## Chapter 2-5, 3강 STT→후처리→TTS 파이프라인 — Gradio 데모

- 목표: 음성 입력→전사→정규화→합성까지 하나의 파이프라인 구성
- 데이터: 수강생 음성 파일(또는 마이크 입력)
- 규칙(강의용): UI는 `gradio` 최소구성, 내부 처리는 `whisper`/`TTS`



#### 구성
- 파이프라인 설계(입력→전사→정규화→합성)
- Whisper 전사(타임스탬프/언어 지정)
- 텍스트 후처리(간단 정규화, 금칙어 제거 예)
- Coqui TTS 합성(문장 단위 분할 합성)
- Gradio UI 구성(업로드→전사→합성→다운로드)



### 0. 환경 준비 및 라이브러리 임포트
- `whisper`, `TTS`, `torchaudio`, `gradio`
- 폰트/경고 설정, 디바이스 확인



In [ ]:
# -*- coding: utf-8 -*-
import os
import re
import warnings
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchaudio

try:
    import whisper
    _HAS_WHISPER = True
except Exception:
    _HAS_WHISPER = False

try:
    from TTS.api import TTS as COQUI_TTS
    _HAS_TTS = True
except Exception:
    _HAS_TTS = False

try:
    import gradio as gr
    _HAS_GRADIO = True
except Exception:
    _HAS_GRADIO = False

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message=r"Glyph.*missing from font.*", category=UserWarning)

plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device:', DEVICE)



### 1. 유틸: 오디오 로딩/저장, 간단 텍스트 정규화
- 16kHz/모노 변환, 문장 분할, 기초 정규화(공백/특수문자 간소화)



In [ ]:
def load_audio_16k_mono(path: str):
    if not os.path.exists(path):
        raise FileNotFoundError('오디오 파일 없음: ' + path)
    wav, sr = torchaudio.load(path)
    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)
    if sr != 16000:
        wav = torchaudio.transforms.Resample(sr, 16000)(wav)
        sr = 16000
    return wav.squeeze(0), sr

_sentence_splitter = re.compile(r"(?<=[.!?。？!])\s+")
_non_korean_keep = re.compile(r"[^0-9A-Za-z가-힣.,!?\s]")

def normalize_text(text: str) -> list[str]:
    text = _non_korean_keep.sub(" ", text)
    text = re.sub(r"\s+", " ", text).strip()
    sentences = re.split(_sentence_splitter, text) if text else []
    # 빈 문장 제거
    return [s for s in sentences if s]



### 2. STT/Whisper: 전사 함수(타임스탬프 포함 옵션)
- 긴 파일은 분할 전사(선택), 여기서는 단일 파일 전사 예시



In [ ]:
def whisper_transcribe(path: str, model_name: str = 'base', language: str | None = None):
    if not _HAS_WHISPER:
        raise ImportError('whisper 설치 필요: pip install -U openai-whisper')
    model = whisper.load_model(model_name, device=DEVICE)
    out = model.transcribe(path, language=language)
    return out['text']



### 3. TTS(Coqui): 문장 단위 합성 및 병합
- 문장별 합성 후 사이에 짧은 무음 패딩을 삽입해 자연스러운 연결



In [ ]:
def tts_sentences(sentences: list[str], speaker_wavs: list[str], language: str = 'ko', pad_ms: int = 200, cleanup: bool = True):
    if not _HAS_TTS:
        raise ImportError('Coqui TTS 설치 필요: pip install TTS')
    tts = COQUI_TTS("tts_models/multilingual/multi-dataset/xtts_v2", gpu=(DEVICE=='cuda'))
    sr = 24000  # xtts 기본 샘플레이트(모델/버전에 따라 다를 수 있습니다)
    print('TTS 샘플레이트(가정):', sr)
    wavs = []
    silence = torch.zeros(int(sr * (pad_ms/1000.0)))
    tmp_files = []
    for s in sentences:
        tmp = f"seg_{abs(hash(s))%10_000}.wav"
        tts.tts_to_file(text=s, speaker_wav=speaker_wavs, language=language, file_path=tmp)
        tmp_files.append(tmp)
        w, r = torchaudio.load(tmp)
        print('segment sr:', r)
        w = w.mean(dim=0) if w.size(0) > 1 else w.squeeze(0)
        if r != sr:
            w = torchaudio.transforms.Resample(r, sr)(w.unsqueeze(0)).squeeze(0)
        wavs.append(w)
        wavs.append(silence.clone())
    out = torch.cat(wavs) if wavs else torch.zeros(1)
    out_path = 'pipeline_tts.wav'
    torchaudio.save(out_path, out.unsqueeze(0), sr)
    if cleanup:
        for f in tmp_files:
            try:
                os.remove(f)
            except Exception:
                pass
    return out_path



### 4. 파이프라인 함수
- 파일 입력→전사→정규화→문장별 합성→다운로드 경로 반환



In [ ]:
def run_pipeline(audio_path: str, language: str = 'ko', model_name: str = 'base', ref_wavs: list[str] | None = None):
    # 1) 전사
    text = whisper_transcribe(audio_path, model_name=model_name, language=language)
    # 2) 텍스트 정규화/분할
    sentences = normalize_text(text)
    if not sentences:
        sentences = [text]
    # 3) 합성
    if not ref_wavs:
        raise ValueError('참조 화자 오디오(ref_wavs)가 필요합니다.')
    out_wav = tts_sentences(sentences, ref_wavs, language=language)
    return text, out_wav



### 5. Gradio UI
- 파일 업로드, 언어/모델 선택, 참조 화자 업로드, 결과 표시



In [ ]:
def app():
    if not _HAS_GRADIO:
        raise ImportError('gradio 설치 필요: pip install gradio')

    def process(audio_file, language, model_name, ref_audio_files):
        if audio_file is None or not audio_file:
            return '오디오 파일을 업로드하세요.', None
        # Gradio는 {'name': path, ...} 또는 str 경로를 줄 수 있음
        in_path = audio_file if isinstance(audio_file, str) else audio_file.get('name')
        refs = []
        if ref_audio_files:
            for f in ref_audio_files:
                p = f if isinstance(f, str) else f.get('name')
                # 16k/모노 보정 파일로 교체 저장
                w, sr = load_audio_16k_mono(p)
                tmp = f"ref_{abs(hash(p))%10_000}.wav"
                torchaudio.save(tmp, w.unsqueeze(0), sr)
                refs.append(tmp)
        if not refs:
            return '참조 화자 오디오를 업로드하세요.', None
        text, out_wav = run_pipeline(in_path, language=language, model_name=model_name, ref_wavs=refs)
        return text, out_wav

    with gr.Blocks() as demo:
        gr.Markdown("# STT→TTS 파이프라인 데모")
        with gr.Row():
            audio_in = gr.Audio(type="filepath", label="입력 오디오")
            ref_in = gr.Files(label="참조 화자 오디오(1~3개)")
        with gr.Row():
            lang = gr.Dropdown(choices=["ko","en","ja"], value="ko", label="언어")
            model = gr.Dropdown(choices=["tiny","base","small"], value="base", label="Whisper 모델")
        run_btn = gr.Button("전사→정규화→합성 실행")
        transcript = gr.Textbox(label="전사 결과")
        audio_out = gr.Audio(label="합성 결과")
        run_btn.click(process, inputs=[audio_in, lang, model, ref_in], outputs=[transcript, audio_out])
    return demo

# 실행 예시 (주피터에서는 아래 두 줄을 수동으로 실행)
# if _HAS_GRADIO:
#     demo = app()
#     demo.launch()

